# WWYZ production lengths

## Setup

In [1]:
import numpy as np
import pandas as pd
import duckdb
from functions import zeroout, get_fatdiag

### Select MRIO version

In [2]:
input, output = 'adb-mrio.parquet', 'lengths.parquet'
# input, output = 'adb-mrio62.parquet', 'lengths62.parquet'
# input, output = 'adb-mrio62-const.parquet', 'lengths62-const.parquet'

### Parameters

In [3]:
sectors = pd.read_excel('../data/raw/sectors.xlsx').drop_duplicates(subset='ind', ignore_index=True)
years = duckdb.sql(f"SELECT DISTINCT t FROM read_parquet('../data/mrio/{input}') ORDER BY t").df()['t']
rows = duckdb.sql(f"SELECT COUNT(*) FROM read_parquet('../data/mrio/{input}')").df()

N = 35                                              # Number of sectors
G = int((rows.iloc[0, 0] / len(years) - 7) / N)     # Number of countries + 1
f = 5                                               # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Decompositions

In [4]:
DF = pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    y = np.sum(Y, axis=1)
    yd = get_fatdiag(Y)
    yf = y - yd
    v = np.where(x != 0, va / x, 0)
    Dx = np.diag(np.where(x != 0, 1 / x, 0))
    A = Z @ Dx
    Ad, Af = zeroout(A, inverse=True), zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)

    X = np.diag(v) @ B @ B @ np.diag(y)
    X_D = np.diag(v) @ Bd @ Bd @ np.diag(yd)
    X_RT = np.diag(v) @ Bd @ Bd @ np.diag(yf)
    Xd_GVC = np.diag(v) @ Bd @ Bd @ Af @ B @ np.diag(y)
    E_GVC = np.diag(v) @ B @ Af @ B @ np.diag(y)
    Xf_GVC = np.diag(v) @ Bd @ Af @ B @ Ad @ B @ np.diag(y)
    VY_D = np.diag(v) @ Bd @ np.diag(yd)
    VY_RT = np.diag(v) @ Bd @ np.diag(yf)
    VY_GVC = np.diag(v) @ Bd @ Af @ B @ np.diag(y)

    DFt = pd.DataFrame({
        't': int(year), 's': np.arange(1, G+1).repeat(N),
        'i': np.tile(sectors['ind'], G), 
        'i5': np.tile(sectors['ind5'], G), 
        'i15': np.tile(sectors['ind15'], G),
        'va': va, 'y': y,
        'Xv': np.sum(X, axis=1),
        'Xv_D': np.sum(X_D, axis=1),
        'Xv_RT': np.sum(X_RT, axis=1),
        'Xvd_GVC': np.sum(Xd_GVC, axis=1),
        'Ev_GVC': np.sum(E_GVC, axis=1),
        'Xvf_GVC': np.sum(Xf_GVC, axis=1),
        'V_D': np.sum(VY_D, axis=1),
        'V_RT': np.sum(VY_RT, axis=1),
        'V_GVC': np.sum(VY_GVC, axis=1),
        'Xy': np.sum(X, axis=0),
        'Xy_D': np.sum(X_D, axis=0),
        'Xy_RT': np.sum(X_RT, axis=0),
        'Xyd_GVC': np.sum(Xd_GVC, axis=0),
        'Ey_GVC': np.sum(E_GVC, axis=0),
        'Xyf_GVC': np.sum(Xf_GVC, axis=0),
        'Y_D': np.sum(VY_D, axis=0),
        'Y_RT': np.sum(VY_RT, axis=0),
        'Y_GVC': np.sum(VY_GVC, axis=0)
    })

    DFagg = DFt.drop(['i', 'i5', 'i15'], axis=1)
    DFagg.insert(2, 'agg', 0)
    DFagg.insert(3, 'i', 0)
    DFagg = DFagg.groupby(['t', 's', 'agg', 'i']).sum().reset_index()

    DF5 = DFt.drop(['i', 'i15'], axis=1)
    DF5.insert(2, 'agg', 5)
    DF5 = DF5.groupby(['t', 's', 'agg', 'i5']).sum().reset_index()
    DF5 = DF5.rename(columns={'i5': 'i'})

    DF15 = DFt.drop(['i', 'i5'], axis=1)
    DF15.insert(2, 'agg', 15)
    DF15 = DF15.groupby(['t', 's', 'agg', 'i15']).sum().reset_index()
    DF15 = DF15.rename(columns={'i15': 'i'})

    DF35 = DFt.drop(['i5', 'i15'], axis=1)
    DF35.insert(2, 'agg', 35)

    DF = pd.concat([DF, DFagg, DF5, DF15, DF35], ignore_index=True)
    
    print(f'{year} done')

2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


In [5]:
APL = pd.DataFrame({
    't': DF['t'], 's': DF['s'], 'agg': DF['agg'], 'i': DF['i'],
    'PLv': DF['Xv'] / DF['va'],
    'PLv_D': DF['Xv_D'] / DF['V_D'],
    'PLv_RT': DF['Xv_RT'] / DF['V_RT'],
    'PLvd_GVC': DF['Xvd_GVC'] / DF['V_GVC'],
    'CBv_GVC': DF['Ev_GVC'] / DF['V_GVC'],
    'PLvf_GVC': DF['Xvf_GVC'] / DF['V_GVC'],
    'PLy': DF['Xy'] / DF['y'],
    'PLy_D': DF['Xy_D'] / DF['Y_D'],
    'PLy_RT': DF['Xy_RT'] / DF['Y_RT'],
    'PLyd_GVC': DF['Xyd_GVC'] / DF['Y_GVC'],
    'CBy_GVC': DF['Ey_GVC'] / DF['Y_GVC'],
    'PLyf_GVC': DF['Xyf_GVC'] / DF['Y_GVC']
})

APL['PLv_GVC'] = APL['PLvd_GVC'] + APL['CBv_GVC'] + APL['PLvf_GVC']
APL['PLy_GVC'] = APL['PLyd_GVC'] + APL['CBy_GVC'] + APL['PLyf_GVC']
APL['GVC_POS'] = APL['PLv_GVC'] / APL['PLy_GVC']

APL.to_parquet(f'../data/{output}', index=False)

### View results

In [6]:
duckdb.sql(f"SELECT * FROM read_parquet('../data/{output}')").df()

,t,s,agg,i,PLv,PLv_D,PLv_RT,PLvd_GVC,CBv_GVC,PLvf_GVC,PLy,PLy_D,PLy_RT,PLyd_GVC,CBy_GVC,PLyf_GVC,PLv_GVC,PLy_GVC,GVC_POS
0,2017,1,0,0,2.291097,1.766394,1.912703,1.766863,1.329694,1.735759,2.064456,1.766394,1.912703,1.959471,1.420106,1.142722,4.832315,4.522299,1.068553
1,2017,2,0,0,1.973771,1.400974,1.595436,1.587044,1.428414,0.876772,2.020244,1.400974,1.595436,1.752809,1.450945,0.736831,3.892230,3.940585,0.987729
2,2017,3,0,0,2.146093,1.391965,1.561568,1.522413,1.455991,0.923992,2.236400,1.391965,1.561568,1.773208,1.426851,0.659961,3.902396,3.860020,1.010978
3,2017,4,0,0,2.252622,1.496919,1.577679,1.563911,1.396315,0.955298,2.189574,1.496919,1.577679,1.818537,1.419644,0.761576,3.915524,3.999756,0.978941
4,2017,5,0,0,1.844668,1.613731,2.104533,1.877153,1.262197,1.118077,1.853104,1.613731,2.104533,1.916715,1.386491,1.073126,4.257427,4.376332,0.972830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,2022,73,35,31,1.404831,1.226389,2.175037,2.706670,1.324536,1.095974,2.599766,2.060891,2.060891,1.765013,1.314484,1.815466,5.127180,4.894963,1.047440
24524,2022,73,35,32,1.227781,1.131713,1.509657,2.542077,1.275089,1.014614,2.611540,2.098434,2.098434,1.762850,1.292671,2.000029,4.831781,5.055550,0.955738
24525,2022,73,35,33,1.409474,1.259167,2.186272,2.916781,1.242070,0.868270,3.090323,2.481298,2.481298,1.773124,1.301718,1.825873,5.027121,4.900716,1.025793
24526,2022,73,35,34,2.562490,1.987876,2.929993,3.547927,1.328422,1.162547,2.925006,2.297026,2.297026,1.881324,1.295535,1.545002,6.038895,4.721862,1.278923
